In [548]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib

matplotlib.rcParams['figure.figsize'] = (20,10)

In [549]:
# Read CSV file into dataframe
# (Bangalore housing data from Kaggle)
# https://www.kaggle.com/datasets/amitabhajoy/bengaluru-house-price-data
path = 'data/Bengaluru_House_Data.csv'
df = pd.read_csv(path)
print("Shape:", df.shape)
df.head()

Shape: (13320, 9)


,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


* Start by exploring the dataset

In [550]:
# Examine some of the columns
for col in ('area_type', 'availability', 'location', 'society'):
    print(df.groupby(col)[col].agg('count').sort_values(ascending=False)[:5], '\n')

area_type
Super built-up  Area    8790
Built-up  Area          2418
Plot  Area              2025
Carpet  Area              87
Name: area_type, dtype: int64 

availability
Ready To Move    10581
18-Dec             307
18-May             295
18-Apr             271
18-Aug             200
Name: availability, dtype: int64 

location
Whitefield         540
Sarjapur  Road     399
Electronic City    302
Kanakpura Road     273
Thanisandra        234
Name: location, dtype: int64 

society
GrrvaGr    80
PrarePa    76
Sryalan    59
Prtates    59
GMown E    56
Name: society, dtype: int64 



In [551]:
# Select and rename relevant columns
df.drop(['area_type', 'society', 'balcony', 'availability'], axis='columns', inplace=True)
df.rename(columns={'total_sqft': 'square_feet', 'size': 'bedrooms', 'bath': 'bathrooms'}, inplace=True)
df.head()

,location,bedrooms,square_feet,bathrooms,price
0,Electronic City Phase II,2 BHK,1056,2.0,39.07
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00
2,Uttarahalli,3 BHK,1440,2.0,62.00
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00
4,Kothanur,2 BHK,1200,2.0,51.00


In [552]:
# Examine the number of null values
df.isnull().sum()

location        1
bedrooms       16
square_feet     0
bathrooms      73
price           0
dtype: int64

In [553]:
# Since there are only a few, drop them
df.dropna(inplace=True)
df.shape

(13246, 5)

In [554]:
# Examine bedrooms values
df['bedrooms'].unique()

array(['2 BHK', '4 Bedroom', '3 BHK', '4 BHK', '6 Bedroom', '3 Bedroom',
       '1 BHK', '1 RK', '1 Bedroom', '8 Bedroom', '2 Bedroom',
       '7 Bedroom', '5 BHK', '7 BHK', '6 BHK', '5 Bedroom', '11 BHK',
       '9 BHK', '9 Bedroom', '27 BHK', '10 Bedroom', '11 Bedroom',
       '10 BHK', '19 BHK', '16 BHK', '43 Bedroom', '14 BHK', '8 BHK',
       '12 Bedroom', '13 BHK', '18 Bedroom'], dtype=object)

In [555]:
import re

# Convert column into numerical values
def extract_num_bedrooms(bedrooms):
    match = re.search('^(\d*) ', bedrooms)
    num_bedrooms = match.group(1) if match is not None else 0
    return int(num_bedrooms)

df['bedrooms'] = df['bedrooms'].apply(extract_num_bedrooms)
df.head()

,location,bedrooms,square_feet,bathrooms,price
0,Electronic City Phase II,2,1056,2.0,39.07
1,Chikka Tirupathi,4,2600,5.0,120.00
2,Uttarahalli,3,1440,2.0,62.00
3,Lingadheeranahalli,3,1521,3.0,95.00
4,Kothanur,2,1200,2.0,51.00


In [556]:
# Explore square feet
df['square_feet'].unique()

array(['1056', '2600', '1440', ..., '1133 - 1384', '774', '4689'],
      dtype=object)

In [557]:
# Convert column into consistent numerical values
unit_map = {'Sq. Meter': 10.7639,
                'Sq. Yards': 9,
                'Acres': 43560,
                'Perch': 272.25,
                'Cents': 435.56,
                'Grounds': 2400,
                'Guntha': 1089}

def extract_square_feet(square_feet):
    if re.match('^\d+\.?\d*$', square_feet):
        return float(square_feet)
    # If the value is a range, return the median
    if re.search(' - ', square_feet):
        match = re.match('^(\d+\.?\d*) - (\d+\.?\d*)$', square_feet)
        return (float(match.group(1)) + float(match.group(2))) / 2
    # Extract the number and type of units and refer to the above mapping
    match = re.match('^(\d+\.?\d*)(\D+)$', square_feet)
    num, unit = float(match.group(1)), match.group(2)
    if unit in unit_map:
        return num * unit_map[unit]
    else:
        return 0

df['square_feet'] = df['square_feet'].apply(extract_square_feet)
df.head()

,location,bedrooms,square_feet,bathrooms,price
0,Electronic City Phase II,2,1056.0,2.0,39.07
1,Chikka Tirupathi,4,2600.0,5.0,120.00
2,Uttarahalli,3,1440.0,2.0,62.00
3,Lingadheeranahalli,3,1521.0,3.0,95.00
4,Kothanur,2,1200.0,2.0,51.00


* Feature Engineering

In [558]:
# Add price per square foot category
df['price_per_sq'] = df['price'] * 100000 / df['square_feet'] #convert from lakh rupees to rupees
df.head()

,location,bedrooms,square_feet,bathrooms,price,price_per_sq
0,Electronic City Phase II,2,1056.0,2.0,39.07,3699.810606
1,Chikka Tirupathi,4,2600.0,5.0,120.00,4615.384615
2,Uttarahalli,3,1440.0,2.0,62.00,4305.555556
3,Lingadheeranahalli,3,1521.0,3.0,95.00,6245.890861
4,Kothanur,2,1200.0,2.0,51.00,4250.000000


In [559]:
# Remove whitespace and capitalization from location category and count results
df['location'] = df['location'].apply(lambda x: x.strip().lower())
location_stats = df.groupby('location')['location'].agg('count').sort_values(ascending=False)
location_stats

location
whitefield               535
sarjapur  road           392
electronic city          304
kanakpura road           266
thanisandra              236
                        ... 
kamanahalli main road      1
kamdhenu nagar             1
1 giri nagar               1
kanakadasa layout          1
zuzuvadi                   1
Name: location, Length: 1282, dtype: int64

In [560]:
# Replace location categories with fewer than 10 instances with "other"
rare_locations = location_stats[location_stats < 10]
df['location'] = df['location'].apply(lambda x: 'other' if x in rare_locations else x)
len(df['location'].unique())

255

In [561]:
g = df.groupby('location')
q1, q3 = g.quantile(.25), g.quantile(.75)
lower_limit, upper_limit = q1 - 1.5 * (q3 - q1), q3 + 1.5 * (q3 - q1)
lower_limit.name = 'location_lower_limit'
df.head()

,location,bedrooms_left,square_feet_left,bathrooms_left,price_left,price_per_sq_left,bedrooms,square_feet,bathrooms,price,price_per_sq
0,electronic city phase ii,2,1056.0,2.0,39.07,3699.810606,NaN,NaN,NaN,NaN,NaN
1,chikka tirupathi,4,2600.0,5.0,120.00,4615.384615,NaN,NaN,NaN,NaN,NaN
2,uttarahalli,3,1440.0,2.0,62.00,4305.555556,NaN,NaN,NaN,NaN,NaN
3,lingadheeranahalli,3,1521.0,3.0,95.00,6245.890861,NaN,NaN,NaN,NaN,NaN
4,kothanur,2,1200.0,2.0,51.00,4250.000000,NaN,NaN,NaN,NaN,NaN
